In [1]:
import os 
import sys
from ase import Atoms

sys.path.append('../pgm/CCS/')
 
from ase import io

from ase.optimize import FIRE
from ase.io import read, write
#import ase.db  
#from ase.units import *
import numpy as np
import pinn
from ase.calculators.dftb import Dftb
from ase.calculators.mixing import LinearCombinationCalculator
import json
from ccs.ase_tools.ccs_ase_calculator import CCS  
 

base_dir=os.getcwd()
os.chdir(base_dir)


if not os.path.isdir("RUN_SIM/"):
    os.mkdir("RUN_SIM/")
os.chdir("RUN_SIM/")
cwd = os.getcwd() 
print('You are here',cwd) 

def setup_model(elements,spin,charge,**kwargs):

    # set up model:
    # 1: DFTB 
    os.environ["DFTB_PREFIX"] = base_dir+"/SKF/"
    DFTB_calc=Dftb(Hamiltonian_SCC='Yes',Hamiltonian_ShellResolvedSCC = 'Yes',Hamiltonian_SCCTolerance = '1.0E-006',Hamiltonian_Mixer= 'Anderson{',Hamiltonian_Mixer_MixingParameter='0.03',Hamiltonian_MaxSCCIterations = '1500',Hamiltonian_Filling = 'MethfesselPaxton{',Hamiltonian_Filling_Temperature = '0.001583407672623195',Hamiltonian_KPointsAndWeights = 'SupercellFolding {1 0 0 0 1 0 0 0 1 0.0 0.0 0.0}') 
    if spin>0:                  
        DFTB_calc.set(Hamiltonian_SpinPolarisation = 'Colinear{') 
        DFTB_calc.set(Hamiltonian_SpinPolarisation_UnpairedElectrons=str(spin))
        DFTB_calc.set(Hamiltonian_SpinConstants_ ='')
        for i in elements: 
            print(i)
            spinparam=get_spindata(i)
            string=f'Hamiltonian_SpinConstants_{i}'  
            print(string)
            DFTB_calc.set(**{string:spinparam})  
    if charge>0: 
        DFTB_calc.set(Hamiltonian_Charge =str(charge))
    print(DFTB_calc.todict())     

                     
    #2: CCS 
    with open(base_dir+'/CCS/CCS_params.json', 'r') as f:
        CCS_params = json.load(f)

    CCS_calc=CCS(CCS_params=CCS_params) 
    #3: PiNN 
    PiNN_calc = pinn.get_calc(base_dir+'/PiNN/') 
    

    # and the calculater becomes 
    calcs =[DFTB_calc,CCS_calc, PiNN_calc]
    weights=[1,1,1]
    calc=LinearCombinationCalculator(calcs, weights) 
    return calc


def get_data(element): 
    if element=='Li':
        maxang="'p'"
    if element=='C':
        maxang="'p'"
    if element=='O':
        maxang="'p'"
    if element=='H':
        maxang="'s'"
    if element=='P':
        maxang="'p'"    
    return maxang                 

def get_spindata(element): 
    if element=='Li':
        spinparam='{-0.019  -0.016  -0.016 -0.027}'
    if element=='C':
        spinparam='{-0.031 -0.025 -0.025 -0.023}'
    if element=='O':
        spinparam='{-0.035 -0.030 -0.030 -0.028}'
    if element=='H':
        spinparam='{-0.072}'
    if element=='Si':
        spinparam='{-0.035 -0.030 -0.030 -0.0280}'   
    if element=='P':
        spinparam='{-0.035 -0.030 -0.030 -0.0280}'  
    return spinparam        



You are here /home/broqvist/Desktop/parautomatik/EC/RUN_SIM


In [7]:
# Geometry optimization
# import structure:
slab = io.read('../DFT_DB.db@0')
elements=['C','H','O'] 
spin=0
charge=0
slab.calc=setup_model(elements, spin, charge)
print(slab.calc)
relax = FIRE(slab, trajectory='opt.traj')
relax.run(fmax=0.05) 

{'Hamiltonian_SCC': 'Yes', 'Hamiltonian_ShellResolvedSCC': 'Yes', 'Hamiltonian_SCCTolerance': '1.0E-006', 'Hamiltonian_Mixer': 'Anderson{', 'Hamiltonian_Mixer_MixingParameter': '0.03', 'Hamiltonian_MaxSCCIterations': '1500', 'Hamiltonian_Filling': 'MethfesselPaxton{', 'Hamiltonian_Filling_Temperature': '0.001583407672623195', 'Hamiltonian_KPointsAndWeights': 'SupercellFolding {1 0 0 0 1 0 0 0 1 0.0 0.0 0.0}'}
LinearCombinationCalculator(Dftb, CCS, PiNN_calc)
      Step     Time          Energy         fmax
FIRE:    0 12:09:56      -61.260366        5.5342
FIRE:    1 12:10:00      -61.589247        2.1162


/home/broqvist/miniconda3/envs/parautomatik/lib/python3.9/site-packages/ase/utils/__init__.py:62: FutureWarning: Please use atoms.cell.rank instead
  warnings.warn(warning)


FIRE:    2 12:10:05      -61.623736        1.5562
FIRE:    3 12:10:11      -61.667185        0.9813
FIRE:    4 12:10:16      -61.694036        1.1849
FIRE:    5 12:10:21      -61.706321        1.6777
FIRE:    6 12:10:24      -61.719978        1.4523
FIRE:    7 12:10:27      -61.732940        0.6974
FIRE:    8 12:10:30      -61.734934        0.8880
FIRE:    9 12:10:32      -61.736375        0.8537
FIRE:   10 12:10:34      -61.738961        0.7872
FIRE:   11 12:10:37      -61.742276        0.6926
FIRE:   12 12:10:39      -61.745773        0.5756
FIRE:   13 12:10:42      -61.748944        0.4428
FIRE:   14 12:10:46      -61.751452        0.3026
FIRE:   15 12:10:50      -61.753164        0.2087
FIRE:   16 12:10:54      -61.754265        0.2360
FIRE:   17 12:10:59      -61.754878        0.2567
FIRE:   18 12:11:03      -61.755258        0.2527
FIRE:   19 12:11:14      -61.755344        0.2478
FIRE:   20 12:11:24      -61.755518        0.2385
FIRE:   21 12:11:30      -61.755780        0.2261


True

In [6]:
!ase db ../DFT_DB.db

id|age|user |formula|calculator| energy|natoms| fmax|pbc|  volume|charge|  mass| smax|magmom
 1| 5M|petbr|C3H4O3 |vasp      |-61.335|    10|3.661|TTT|8400.000| 0.000|88.062|0.000| 0.000
 2| 5M|petbr|C3H4O3 |vasp      |-56.749|    10|4.304|TTT|8400.000| 0.000|88.062|0.001| 0.489
 3| 5M|petbr|C3H4O3 |vasp      |-55.991|    10|4.085|TTT|8400.000| 0.000|88.062|0.001| 0.523
 4| 5M|petbr|C3H4O3 |vasp      |-55.893|    10|5.411|TTT|8400.000| 0.000|88.062|0.001| 0.398
 5| 5M|petbr|C3H4O3 |vasp      |-55.916|    10|7.592|TTT|8400.000| 0.000|88.062|0.001| 0.044
 6| 5M|petbr|C3H4O3 |vasp      |-56.213|    10|7.127|TTT|8400.000| 0.000|88.062|0.001| 0.551
 7| 5M|petbr|C3H4O3 |vasp      |-56.231|    10|6.536|TTT|8400.000| 0.000|88.062|0.001| 0.521
 8| 5M|petbr|C3H4O3 |vasp      |-55.914|    10|5.038|TTT|8400.000| 0.000|88.062|0.001| 0.906
 9| 5M|petbr|C3H4O3 |vasp      |-55.546|    10|5.810|TTT|8400.000| 0.000|88.062|0.001| 1.176
10| 5M|petbr|C3H4O3 |vasp      |-56.225|    10|3.393|TTT|8400.000| 0.0

In [ ]:
# MD 
from ase import units
from ase.io import read, write
from ase.io.trajectory import Trajectory
from ase.md import MDLogger
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase.md.nptberendsen import NPTBerendsen
from ase.md.nvtberendsen import NVTBerendsen
from ase.md.verlet import VelocityVerlet

rng = np.random.default_rng(10)
atoms = read("opt.traj")
atoms.set_calculator(calc)
MaxwellBoltzmannDistribution(atoms, 873*units.kB, rng=rng)
dt = 0.5 * units.fs
steps = int(10)
dyn = VelocityVerlet(atoms, timestep=dt)
interval = int(1)
dyn.attach(MDLogger(dyn, atoms, 'output.log', mode="a"), interval=interval)
dyn.attach(Trajectory('output.traj', 'a', atoms).write, interval=interval)
dyn.run(steps)

In [ ]:
write_dftb_velocities(atoms, 'velocities.txt')

atoms.calc = calculator_NVE
atoms.get_potential_energy()  # run NVE ensemble using DFTB+'s own driver
atoms = read('geo_end.gen')
write('after_NVE.xyz', atoms)

read_dftb_velocities(atoms, filename='geo_end.xyz')
write_dftb_velocities(atoms, 'velocities.txt')
os.system('mv geo_end.xyz geo_end_NVE.xyz')

atoms.calc = calculator_NVT
atoms.get_potential_energy()  # run NVT ensemble using DFTB+'s own driver
atoms = read('geo_end.gen')
write('after_NVT.xyz', atoms)

read_dftb_velocities(atoms, filename='geo_end.xyz')
write_dftb_velocities(atoms, 'velocities.txt')
os.system('mv geo_end.xyz geo_end_NVT.xyz')
